## Image Colorization Interface

In [10]:
#Import Libraries
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
import skimage.io
from skimage.io import imsave, imshow
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import h5py
import matplotlib
import matplotlib.gridspec as gridspec
#import cv2
from tensorflow.keras.layers import Input,Dense,Reshape,Conv2D,Dropout,multiply,Dot,Concatenate,subtract,ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization,LeakyReLU,Flatten
from tensorflow.keras.layers import Conv2DTranspose as Deconv2d
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from google.colab import files
from tensorflow.keras import backend as K
import sklearn
from sklearn.utils import shuffle

In [11]:
print('Versions: \n')
print('Tensorflow',tf.__version__)
print('Keras',tf.keras.__version__)
print('Scikit-Image',skimage.__version__)
print('Scikit-Learn',sklearn.__version__)
print('Numpy',np.__version__)
print('Pandas',pd.__version__)
print('PIL',Image.__version__)

Versions: 

Tensorflow 2.1.0
Keras 2.2.4-tf
Scikit-Image 0.17.2
Scikit-Learn 0.23.2
Numpy 1.17.4
Pandas 0.25.3
PIL 7.2.0


### Due to using Google Colab for training, the saved model can't be loaded as Colab uses Tensorflow 2.4.x and Anaconda only has support upto 2.1

### Therefore the only way to use model for Interface is by constructing it again and using the saved weights

In [20]:
img_size = 128

### Generaotr Architecture 

In [21]:

def gen_model():

    inputs = tf.keras.layers.Input( shape=( img_size , img_size , 1 ) )

    c1 = tf.keras.layers.Conv2D( 16 , kernel_size=( 3 , 3 ) , strides=1 )( inputs )
    c1 = tf.keras.layers.LeakyReLU()( c1 )
    c1 = tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=1)( c1 )
    c1 = tf.keras.layers.LeakyReLU()( c1 )

    c2 = tf.keras.layers.Conv2D( 64 , kernel_size=( 3 , 3 ) , strides=1)( c1 )
    c2 = tf.keras.layers.LeakyReLU()( c2 )
    c2 = tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=1 )( c2 )
    c2 = tf.keras.layers.LeakyReLU()( c2 )

    c3 = tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 )( c2 )
    c3 = tf.keras.layers.LeakyReLU()( c3 )

    bottleneck = tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' , padding='same' )( c3 )

    concat_1 = tf.keras.layers.Concatenate()( [ bottleneck , c3 ] )
    ct3 = tf.keras.layers.Conv2DTranspose( 256 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_1 )

    concat_2 = tf.keras.layers.Concatenate()( [ ct3 , c2 ] )
    ct2 = tf.keras.layers.Conv2DTranspose( 128 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( concat_2 )
    ct2 = tf.keras.layers.Conv2DTranspose( 64 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu' )( ct2 )

    concat_3 = tf.keras.layers.Concatenate()( [ ct2 , c1 ] )
    ct1 = tf.keras.layers.Conv2DTranspose( 32 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( concat_3 )
    ct1 = tf.keras.layers.Conv2DTranspose( 16 , kernel_size=( 3 , 3 ) , strides=1 , activation='relu')( ct1 )

    ct1 = tf.keras.layers.Conv2DTranspose( 3 , kernel_size=( 1 , 1 ) , strides=1 , activation='relu')( ct1 )

    model = tf.keras.models.Model( inputs , ct1 )
    return model


### Discriminator Architecture

In [22]:
def disc_model():
    layers = [
        tf.keras.layers.Conv2D( 32 , kernel_size=( 3 , 3 ) , strides=2, input_shape=( 128 , 128 , 3 ) ),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Conv2D( 64 , kernel_size=( 3, 3 ) , strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv2D( 128 , kernel_size=( 3 , 3 ) , strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D( 256 , kernel_size=( 3 , 3 ) , strides=2, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense( 1 , activation='sigmoid' ) 
    ]
    model = tf.keras.models.Sequential( layers )
    return model


In [23]:
generator = gen_model()
discriminator = disc_model()

In [24]:
generator.load_weights('img_data/saves/gen_weights2.h5')
discriminator.load_weights('img_data/saves/dis_weights2.h5')

In [25]:
#from tensorflow.keras.models import load_model
#generatorx = load_model('image_data/saves/generatorxx.h5')

## Interface 

In [36]:
import numpy as np
import cv2
import PySimpleGUI as sg
import os.path


generator.load_weights('saves/gen_weights2.h5')
discriminator.load_weights('saves/dis_weights2.h5')


def colorize_image(image_filename=None):
    img1_color = []
    test_image = cv2.imread(image_filename)
    test_image = cv2.resize(test_image, (128, 128))
    img1=img_to_array(test_image)
    img1_color.append(img1)
    img1_color = np.array(img1_color, dtype=float)
    img1_color = rgb2lab(1.0/255*img1_color)[:,:,:,0]
    img1_color = img1_color.reshape(img1_color.shape+(1,))
    img2_color = np.float32(img1_color)
    y = generator(img2_color).numpy()
    image = Image.fromarray( ( y[0] ).astype( 'uint8' ) )
    image = np.asarray( image )
    return test_image, image


# --------------------------------- The GUI ---------------------------------

# First the window layout...2 columns

left_col = [[sg.Text('Folder'), sg.In(size=(25,1), enable_events=True ,key='-FOLDER-'), sg.FolderBrowse()],
            [sg.Listbox(values=[], enable_events=True, size=(40,20),key='-FILE LIST-')]]

images_col = [[sg.Text('Input file:'), sg.In(enable_events=True, key='-IN FILE-'), sg.FileBrowse()],
              [sg.Button('Colorize Photo', key='-PHOTO-'), sg.Button('Save File', key='-SAVE-'), sg.Button('Exit')],
              [sg.Image(filename='', key='-IN-'), sg.Image(filename='', key='-OUT-')],]
# ----- Full layout -----
layout = [[sg.Column(left_col), sg.VSeperator(), sg.Column(images_col)]]

# ----- Make the window -----
window = sg.Window('Image Colorizer', layout, grab_anywhere=True)

# ----- Run the Event Loop -----
prev_filename = colorized = cap = None
while True:
    event, values = window.read()
    if event in (None, 'Exit'):
        break
    if event == '-FOLDER-':         # Folder selection
        folder = values['-FOLDER-']
        img_types = (".png", ".jpg", "jpeg", ".tiff", ".bmp") # type of image files suppot
        # get list of files in folder
        try:
            flist0 = os.listdir(folder)
        except:
            continue
        fnames = [f for f in flist0 if os.path.isfile(
            os.path.join(folder, f)) and f.lower().endswith(img_types)]
        window['-FILE LIST-'].update(fnames)
    elif event == '-FILE LIST-':    # A file was chosen from the listbox
        try:
            filename = os.path.join(values['-FOLDER-'], values['-FILE LIST-'][0])
            image = cv2.imread(filename)
            imageS = cv2.resize(image, (128, 128))
            window['-IN-'].update(data=cv2.imencode('.png', imageS)[1].tobytes())
            window['-OUT-'].update(data='')
            window['-IN FILE-'].update('')
            imageO, colorized = colorize_image(imageS)
            window['-OUT-'].update(data=cv2.imencode('.png', colorized)[1].tobytes())
        except:
            continue
    elif event == '-PHOTO-':        # Colorize photo button clicked
        try:
            if values['-IN FILE-']:
                filename = values['-IN FILE-']
            elif values['-FILE LIST-']:
                filename = os.path.join(values['-FOLDER-'], values['-FILE LIST-'][0])
            else:
                continue
            image, colorized = colorize_image(filename)
            window['-IN-'].update(data=cv2.imencode('.png', image)[1].tobytes())
            window['-OUT-'].update(data=cv2.imencode('.png', colorized)[1].tobytes())
        except:
            continue
    elif event == '-IN FILE-':      # A single filename was chosen
        filename = values['-IN FILE-']
        if filename != prev_filename:
            prev_filename = filename
            try:
                image = cv2.imread(filename)
                imageS = cv2.resize(image, (128, 128))
                window['-IN-'].update(data=cv2.imencode('.png', imageS)[1].tobytes())
                
            except:
                continue
    elif event == '-SAVE-' and colorized is not None:   # Clicked the Save File button
        filename = sg.popup_get_file('Save colorized image\nEnter a image name and same extension as original for saving', save_as=True)
        try:
            if filename:
                cv2.imwrite(filename, colorized)
                sg.popup_quick_message('Image saved', background_color='black', text_color='white', font='Cambria')
        except:
            sg.popup_quick_message('Failed!', background_color='black', text_color='white', font='Cambria')
# ----- Exit program -----
window.close()